# Indexación y selección de datos

Analizamos en detalle los métodos y herramientas para acceder, establecer y modificar valores en matrices NumPy.
Estos incluían la indexación (e.g., ``arr[2, 1]``), slicing (e.g., ``arr[:, 1:5]``),enmascaramiento (e.g., ``arr[arr > 0]``), indexación elegante (e.g., ``arr[0, [1, 5]]``), y combinaciones de los mismos (e.g., ``arr[:, [1, 5]]``).
**Aquí veremos medios similares para acceder y modificar valores en los objetos ``Series`` y ``DataFrame`` de Pandas.**
Si ha utilizado los patrones NumPy, los patrones correspondientes en Pandas le resultarán muy familiares, aunque hay algunas peculiaridades que debe tener en cuenta.

Comenzaremos con el caso simple del objeto ``Series`` unidimensional y luego pasaremos al objeto bidimensional más complicado ``DataFrame``.

## Selección de datos en serie

Como vimos en la sección anterior, un objeto **``Series`` actúa en muchos sentidos como una matriz NumPy unidimensional y en muchos sentidos como un diccionario estándar de Python.**
Si tenemos en cuenta estas dos analogías superpuestas, nos ayudará a comprender los patrones de indexación y selección de datos en estas matrices.

### Serie como diccionario

Como un diccionario, el objeto ``Series`` proporciona una asignación de una colección de claves a una colección de valores:

In [3]:
import pandas as pd
data = pd.Series([0.25, 0.5, 0.75, 1.0],
                 index=['a', 'b', 'c', 'd'])
data

a    0.25
b    0.50
c    0.75
d    1.00
dtype: float64

In [2]:
data['b']

0.5

También podemos usar expresiones y métodos de Python similares a un diccionario para examinar las claves/índices y los valores:

In [3]:
'b' in data

True

In [4]:
data.keys()

Index(['a', 'b', 'c', 'd'], dtype='object')

In [5]:
data.index

Index(['a', 'b', 'c', 'd'], dtype='object')

In [7]:
data.values

array([0.25, 0.5 , 0.75, 1.  ])

In [6]:
list(data.items())

[('a', 0.25), ('b', 0.5), ('c', 0.75), ('d', 1.0)]

Los objetos ``Series`` pueden incluso modificarse con una sintaxis similar a la de un diccionario.
Así como puedes extender un diccionario asignándolo a una nueva clave, puedes extender una ``Serie`` asignándole un nuevo valor de índice:

In [8]:
data['e'] = 1.25
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

Esta fácil mutabilidad de los objetos es una característica conveniente: bajo el capó, Pandas está tomando decisiones sobre el diseño de la memoria y la copia de datos que podrían necesitar realizarse; el usuario generalmente no necesita preocuparse por estos problemas.

### Serie como matriz unidimensional

Una ``Serie`` se basa en esta interfaz similar a un diccionario y proporciona selección de elementos de estilo matriz a través de los mismos mecanismos básicos que las matrices NumPy, es decir, *slices*, *enmascaramiento* e *indexación elegante*.
Ejemplos de estos son los siguientes:

In [9]:
# slicing by explicit index
data['a':'c']

a    0.25
b    0.50
c    0.75
dtype: float64

In [10]:
# slicing by implicit integer index
data[0:2]

a    0.25
b    0.50
dtype: float64

In [11]:
data

a    0.25
b    0.50
c    0.75
d    1.00
e    1.25
dtype: float64

In [14]:
(data > 0.3) & (data < 0.8)

a    False
b     True
c     True
d    False
e    False
dtype: bool

In [15]:
# masking o filtrado
data[(data > 0.3) & (data < 0.8)]

b    0.50
c    0.75
dtype: float64

In [19]:
# fancy indexing con doble corchete
data[['a', 'e']]

a    0.25
e    1.25
dtype: float64

Entre estos, el corte puede ser la fuente de mayor confusión.
**Tenga en cuenta que al cortar con un índice explícito (i.e., ``data['a':'c']``), el índice final está *incluido* en el corte, mientras que cuando se corta con un índice implícito (i.e., ``data[0:2]``), el índice final está *excluido* del segmento.**

### Indexadores: loc, iloc

Estas convenciones de división e indexación pueden ser fuente de confusión.
Por ejemplo, si su ``Serie`` tiene un índice entero explícito, una operación de indexación como **``datos[1]`` utilizará los índices explícitos, mientras que una operación de división como ``datos[1:3] `` utilizará el índice implícito de estilo Python.**

In [20]:
data = pd.Series(['a', 'b', 'c'], index=[1, 3, 5])
data

1    a
3    b
5    c
dtype: object

In [21]:
# explicit index when indexing
data[1]

'a'

In [22]:
# implicit index when slicing con indexacion natural
data[1:3]

3    b
5    c
dtype: object

Debido a esta posible confusión en el caso de índices enteros, Pandas proporciona algunos atributos *indexador* especiales que exponen explícitamente ciertos esquemas de indexación.
Estos no son métodos funcionales, sino atributos que exponen una interfaz de corte particular a los datos de la ``Serie``.

Primero, el atributo **``loc`` permite indexar y dividir que siempre hace referencia al índice explícito:**

In [23]:
data

1    a
3    b
5    c
dtype: object

In [24]:
data.loc[5] # respeta los índices que le hemos indicado.

'c'

In [28]:
print(data[1:3])
print(data.loc[1:3]) #desde el indice 1 hasta el indice 3 (incluido) por el .loc

3    b
5    c
dtype: object
1    a
3    b
dtype: object


El atributo ``iloc`` permite indexar y dividir que siempre hace referencia al índice implícito de estilo Python:

In [29]:
data

1    a
3    b
5    c
dtype: object

In [30]:
data.iloc[1] # en función del índice numérico que viene por defecto.

'b'

In [31]:
data.iloc[1:2]

3    b
dtype: object

Un tercer atributo de indexación, ``ix``, es un híbrido de los dos, y para objetos ``Series`` es equivalente a la indexación estándar basada en ``[]``.
El propósito del indexador ``ix`` se hará más evidente en el contexto de los objetos ``DataFrame``, que discutiremos en un momento.

Un principio rector del código Python es que "lo explícito es mejor que lo implícito".
La naturaleza explícita de ``loc`` e ``iloc`` los hace muy útiles para mantener un código limpio y legible; especialmente en el caso de índices enteros, **recomiendo usarlos para hacer que el código sea más fácil de leer y comprender, y para evitar errores sutiles debido a la convención mixta de indexación/corte.**

## Selección de datos en DataFrame

Recuerde que un ``DataFrame`` actúa en muchos sentidos como una matriz estructurada o bidimensional, y en otros aspectos como un diccionario de estructuras de ``Series`` que comparten el mismo índice.
Puede ser útil tener en cuenta estas analogías a medida que exploramos la selección de datos dentro de esta estructura.

### Marco de datos como diccionario

La primera analogía que consideraremos es el ``DataFrame`` como un diccionario de objetos ``Series`` relacionados.
Volvamos a nuestro ejemplo de áreas y poblaciones de estados:

In [4]:
area = pd.Series({'California': 423967, 'Texas': 695662,
                  'New York': 141297, 'Florida': 170312,
                  'Illinois': 149995})
pop = pd.Series({'California': 38332521, 'Texas': 26448193,
                 'New York': 19651127, 'Florida': 19552860,
                 'Illinois': 12882135})
# cities = pd.Series({'California': "Sacramento", 'Texas': "Otra ciudad 1",
#                  'New York': "Albany", 'Florida': "Miami",
#                  'Illinois': "Otra ciudad 2"})
data = pd.DataFrame({'area':area, 'population':pop})
data

,area,population
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [ ]:
# data.reset_index()

Se puede acceder a las ``Series`` individuales que componen las columnas del ``DataFrame`` mediante la indexación estilo diccionario del nombre de la columna:

In [38]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

De manera equivalente, podemos usar el acceso de estilo de atributo con nombres de columna que sean cadenas:

In [37]:
data.population

California    38332521
Texas         26448193
New York      19651127
Florida       19552860
Illinois      12882135
Name: population, dtype: int64

In [39]:
data.pop

<bound method DataFrame.pop of               area  population
California  423967    38332521
Texas       695662    26448193
New York    141297    19651127
Florida     170312    19552860
Illinois    149995    12882135>

In [40]:
data

,area,population
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


Este acceso a la columna de estilo de atributo en realidad accede exactamente al mismo objeto que el acceso de estilo de diccionario:

In [41]:
data.area is data['area']

True

Aunque esta es una abreviatura útil, tenga en cuenta que no funciona en todos los casos.
Por ejemplo, **si los nombres de las columnas no son cadenas, o si los nombres de las columnas entran en conflicto con los métodos del ``DataFrame``, este acceso de estilo de atributo no es posible.**
Por ejemplo, ``DataFrame`` tiene un método ``pop()``, por lo que ``data.pop`` apuntará a esto en lugar de a la columna ``"pop"``:

In [42]:
data.pop is data['pop']

KeyError: 'pop'

En particular, debe evitar la tentación de intentar la asignación de columnas mediante atributos (es decir, usar ``data['pop'] = z`` en lugar de ``data.pop = z``).

Al igual que con los objetos ``Series`` discutidos anteriormente, esta sintaxis estilo diccionario también se puede usar para modificar el objeto, en este caso agregando una nueva columna:

In [43]:
data

,area,population
California,423967,38332521
Texas,695662,26448193
New York,141297,19651127
Florida,170312,19552860
Illinois,149995,12882135


In [ ]:
# for pop in data['population'].values:
#     for area in data['area'].values:
#         print("population", pop)
#         print("area", area)
#         print("density", pop / area)

In [5]:
data['density'] = data['population'] / data['area']
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Esto muestra una vista previa de la sintaxis sencilla de la aritmética elemento por elemento entre objetos ``Serie``; profundizaremos en esto más adelante.

### DataFrame como matriz bidimensional

Como se mencionó anteriormente, también podemos ver el ``DataFrame`` como una matriz bidimensional mejorada.
Podemos examinar la matriz de datos subyacente sin procesar usando el atributo ``valores``:

In [45]:
data.values

array([[4.23967000e+05, 3.83325210e+07, 9.04139261e+01],
       [6.95662000e+05, 2.64481930e+07, 3.80187404e+01],
       [1.41297000e+05, 1.96511270e+07, 1.39076746e+02],
       [1.70312000e+05, 1.95528600e+07, 1.14806121e+02],
       [1.49995000e+05, 1.28821350e+07, 8.58837628e+01]])

Con esta imagen en mente, se pueden realizar muchas observaciones familiares similares a matrices en el propio ``DataFrame``.
**Por ejemplo, podemos transponer el ``DataFrame`` completo para intercambiar filas y columnas:**

In [46]:
data.T

,California,Texas,New York,Florida,Illinois
area,4.239670e+05,6.956620e+05,1.412970e+05,1.703120e+05,1.499950e+05
population,3.833252e+07,2.644819e+07,1.965113e+07,1.955286e+07,1.288214e+07
density,9.041393e+01,3.801874e+01,1.390767e+02,1.148061e+02,8.588376e+01


Sin embargo, cuando se trata de indexar objetos ``DataFrame``, está claro que la indexación de columnas al estilo de un diccionario impide nuestra **capacidad de tratarlo simplemente como una matriz NumPy.**
En particular, al pasar un único índice a una matriz se accede a una fila:

In [48]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [49]:
data.values[0]

array([4.23967000e+05, 3.83325210e+07, 9.04139261e+01])

y al pasar un único "índice" a un ``DataFrame`` se accede a una columna:

In [50]:
data['area']

California    423967
Texas         695662
New York      141297
Florida       170312
Illinois      149995
Name: area, dtype: int64

Por tanto, para la indexación de estilo matriz, necesitamos otra convención.
Aquí Pandas vuelve a utilizar los indexadores ``loc``, ``iloc`` y ``ix`` mencionados anteriormente.
Usando el indexador ``iloc``, podemos indexar la matriz subyacente como si fuera una matriz NumPy simple (usando el índice implícito estilo Python), pero el índice ``DataFrame`` y las etiquetas de las columnas se mantienen en el resultado:

In [51]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [54]:
data.iloc[:4:2, 1:3]

,population,density
California,38332521,90.413926
New York,19651127,139.076746


De manera similar, usando el indexador ``loc`` podemos indexar los datos subyacentes en un estilo similar a una matriz pero usando el índice explícito y los nombres de las columnas:

In [55]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [59]:
data.loc['Illinois':'Texas':-1, 'population':'density'] #-1 cambia el orden, lo pone al reves

,population,density
Illinois,12882135,85.883763
Florida,19552860,114.806121
New York,19651127,139.076746
Texas,26448193,38.018740


**<font color='red'>IX ESTA DEPRECADO</font>**

El indexador ``ix`` permite un híbrido de estos dos enfoques:

In [57]:
pd.__version__

'2.1.1'

Tenga en cuenta que para índices enteros, el indexador ``ix`` está sujeto a las mismas fuentes potenciales de confusión que las analizadas para los objetos ``Series`` indexados con enteros.

Cualquiera de los patrones familiares de acceso a datos de estilo NumPy se puede utilizar dentro de estos indexadores.
Por ejemplo, en el indexador ``loc`` podemos combinar enmascaramiento e indexación elegante como se muestra a continuación:

In [60]:
data

,area,population,density
California,423967,38332521,90.413926
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [63]:
data.loc[data.density > 100, ['population', 'density']] #filtrado del dataframe

,population,density
New York,19651127,139.076746
Florida,19552860,114.806121


Cualquiera de estas convenciones de indexación también se puede utilizar para establecer o modificar valores; Esto se hace de la forma estándar a la que quizás esté acostumbrado al trabajar con NumPy:

In [6]:
data.iloc[0, 2] = 90
data

,area,population,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Para desarrollar su fluidez en la manipulación de datos de Pandas, le sugiero dedicar algo de tiempo a un ``DataFrame`` simple y explorar los tipos de indexación, división, enmascaramiento e indexación sofisticada que permiten estos diversos enfoques de indexación.

### Convenciones de indexación adicionales

Hay un par de convenciones de indexación adicionales que pueden parecer contradictorias con la discusión anterior, pero que, sin embargo, pueden resultar muy útiles en la práctica.
**Primero, mientras que *indexar* se refiere a columnas, *rebanar* se refiere a filas:**

In [7]:
data['Florida':'Illinois']

,area,population,density
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


In [8]:
data.loc[:, 'area':'density']

,area,population,density
California,423967,38332521,90.000000
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121
Illinois,149995,12882135,85.883763


Estos sectores también pueden referirse a filas por número en lugar de por índice:

In [9]:
data[1:3]

,area,population,density
Texas,695662,26448193,38.018740
New York,141297,19651127,139.076746


De manera similar, las operaciones de enmascaramiento directo también se interpretan por filas en lugar de por columnas:

In [10]:
data[data.density > 100]

,area,population,density
New York,141297,19651127,139.076746
Florida,170312,19552860,114.806121


Estas dos convenciones son sintácticamente similares a las de una matriz NumPy y, aunque es posible que no encajen exactamente en el molde de las convenciones de Pandas, son bastante útiles en la práctica.